In [1]:
import pandas as pd
import numpy as np

In [5]:
print(pd.__version__, np.__version__, sep= '; ')

0.18.0; 1.10.4


In [121]:
tr = pd.read_csv('/Users/evgenysidorov/Documents/Kaggle/bnp/data/train.csv')

In [10]:
print(tr.shape)
tr.head(1)

(114321, 133)


,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.0,1.98978,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144


Типы переменных, для флотов уже есть идея из слака привести их назад к [0, 20].
Для текстовых переменных - следую небольшому анализу с форума. Есть большие корреляции

In [11]:
tr.dtypes.value_counts()

float64    108
object      19
int64        6
dtype: int64

In [122]:
del tr['v91'] # 1-в-1 соответствие с какой-то другой колонкой

In [21]:
# колонка v47 более широкая чем v110, это как департаменты и подтепартменты в уолмарте были
(tr['v47']+tr['v110']).value_counts()

CB    55425
IA    39071
EA     5301
FA     4322
GA     3946
DC     3157
JA     3010
BC       50
AA       38
HB        1
dtype: int64

In [23]:
(tr['v71']+tr['v75']).value_counts() # можно объединить в одну колонку, хотя потеряется немного информации.

FD    75086
BB    30247
CB     8944
IC        9
IA        7
FC        6
BC        4
BA        4
GA        3
GC        2
DC        1
AA        1
LC        1
FA        1
CA        1
FB        1
KA        1
CD        1
CC        1
dtype: int64

По ссылке есть уже найденные сильно скоррелированные переменные https://www.kaggle.com/c/bnp-paribas-cardif-claims-management/forums/t/19240/analysis-of-duplicate-variables-correlated-variables-large-post <br>
Нужно будет сделать для них PCA, например

In [ ]:
I've also discovered that v72 is the exact sum of v129 v38 and v62.

# Пора модель гонять!

In [32]:
tr.head(1)

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.0,1.98978,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144


## Что нужно сделать в будущем
    1. Почистить скоррелированность и связанность (с форума взять инфу)
    2. Отскейлить(понять как это делается) и брут-форсом вставлять комбинации. Почистить лассо или важностью фичей
    3. Избавиться от шума в переменных? - вычесть ошибку каки-то образом
    4. Энкодинг
    5. Выделить группы фичей 
    6. Сделать счетчики - число NA, число значений больше среднего, какого-то квантиля

In [39]:
from sklearn.linear_model import LassoCV

In [40]:
from sklearn.cross_validation import StratifiedShuffleSplit

In [41]:
from sklearn.preprocessing import LabelEncoder

In [79]:
ss = StratifiedShuffleSplit(tr['target'].values, n_iter = 1, test_size = 0.3, random_state=24)

In [123]:
for i in list(tr.columns):
    if tr[i].dtype == 'O':
        tr.loc[:, i] = pd.factorize(tr[i])[0]

In [103]:
from sklearn.preprocessing import StandardScaler
st_sc = StandardScaler()
tr.iloc[:, 2:] = st_sc.fit_transform(tr.iloc[:, 2:] )

In [105]:
l_cv = LassoCV(eps=0.0001, 
               n_alphas=1000, 
               alphas=None, 
               fit_intercept=True, 
               normalize=False, 
               precompute='auto', 
               max_iter=10000, 
               tol=0.0001, 
               copy_X=True, 
               cv=None, 
               verbose=False, 
               n_jobs=-1, 
               positive=False, 
               random_state=22, 
               selection='cyclic')

In [106]:
tr.head(1)

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,0.554085,1.042047,0.149675,0.748987,0.655742,0.942241,1.240098,-0.172595,...,1.117302,0.153245,-0.036475,-1.536665,0.940745,0.708351,0.752294,-0.447372,-0.007953,1.442652


In [126]:
tr['num_na'] = tr.apply(lambda x: x.shape[0] - x.count(), axis = 1)

In [131]:
tr_updated = pd.concat([tr, tr.iloc[:, 2:50].apply(lambda x: x.apply(lambda y: y<x.mean()), axis = 0)], axis = 1)

In [140]:
tr_updated.iloc[:, 133:] = tr_updated.iloc[:, 133:].astype(int)

In [138]:
tr.fillna(-1, inplace=True)

In [107]:
l_cv.fit(tr.iloc[:, 2:], tr.iloc[:, 1])

/Users/evgenysidorov/anaconda/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


LassoCV(alphas=None, copy_X=True, cv=None, eps=0.0001, fit_intercept=True,
    max_iter=10000, n_alphas=1000, n_jobs=-1, normalize=False,
    positive=False, precompute='auto', random_state=22, selection='cyclic',
    tol=0.0001, verbose=False)

In [108]:
l_cv.alpha_

9.8093571446421105e-05

In [118]:
l = list()
for j in range(len(l_cv.coef_)):
    if abs(l_cv.coef_[j])>0.001:
        print(list(tr.columns)[j])
        l.append(list(tr.columns)[j])
# l.remove('target')

ID
target
v1
v2
v4
v6
v7
v8
v10
v12
v13
v14
v15
v16
v17
v22
v24
v25
v26
v28
v29
v34
v36
v37
v43
v44
v45
v48
v52
v53
v54
v57
v62
v63
v64
v68
v69
v72
v77
v79
v80
v82
v83
v87
v90
v92
v96
v97
v100
v103
v107
v108
v109
v110
v112
v113
v115
v117
v122
v123
v127
v129


In [72]:
import xgboost as xgb

In [143]:
for train_index, val_index in ss:
    dtrain = xgb.DMatrix(tr_updated.iloc[train_index,2:].as_matrix(), label = tr.iloc[train_index, 1].values)
    dvalid = xgb.DMatrix(tr_updated.iloc[val_index,2:].as_matrix(), label = tr.iloc[val_index, 1].values)
    params = {"objective": "binary:logistic",
   "booster": "gbtree",
      "learning_rate": 0.1,
      "max_depth": 8,
      "subsample": 0.8,
      "colsample_bytree":0.7,
      "silent": 1,
      'min_child_weight': 11,
      'eval_metric':'logloss',
    'seed':22,
    'nthread':2}
#     params = { 
#    "objective": "binary:logistic",
#    "booster": "gbtree",
#    "eval_metric": "logloss",
#    "eta": 0.01, # 0.06, #0.01,
#    #"min_child_weight": 240,
#    "subsample": 0.75,
#    "colsample_bytree": 0.68,
#    "max_depth": 11
# }


    num_trees = 100
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm = xgb.train(params, 
                dtrain, 
                num_trees, 
                        evals=watchlist, 
#                         early_stopping_rounds=50, 
#                         feval=rmspe_xg, 
#                     verbose_eval=True
               )

[0]	eval-logloss:0.661106	train-logloss:0.660323
[1]	eval-logloss:0.631145	train-logloss:0.629617
[2]	eval-logloss:0.608004	train-logloss:0.605314
[3]	eval-logloss:0.590643	train-logloss:0.587228
[4]	eval-logloss:0.572815	train-logloss:0.568403
[5]	eval-logloss:0.559878	train-logloss:0.554730
[6]	eval-logloss:0.546631	train-logloss:0.540595
[7]	eval-logloss:0.535462	train-logloss:0.528554
[8]	eval-logloss:0.526695	train-logloss:0.518714
[9]	eval-logloss:0.518639	train-logloss:0.509765
[10]	eval-logloss:0.513452	train-logloss:0.503671
[11]	eval-logloss:0.508138	train-logloss:0.497674
[12]	eval-logloss:0.503566	train-logloss:0.492435
[13]	eval-logloss:0.499303	train-logloss:0.487112
[14]	eval-logloss:0.496009	train-logloss:0.482952
[15]	eval-logloss:0.493434	train-logloss:0.479699
[16]	eval-logloss:0.490730	train-logloss:0.475844
[17]	eval-logloss:0.488789	train-logloss:0.473261
[18]	eval-logloss:0.486063	train-logloss:0.469722
[19]	eval-logloss:0.483800	train-logloss:0.466434
[20]	eval-

## External solution

In [36]:
import pandas as pd

import numpy as np
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import ensemble


def find_delimiter(df, col):
    """
    Function that trying to find an approximate delimiter used for scaling.
    So we can undo the feature scaling.
    """
    vals = df[col].dropna().sort_values().round(8)
    vals = pd.rolling_apply(vals, 2, lambda x: x[1] - x[0])
    vals = vals[vals > 0.000001]
    return vals.value_counts().idxmax() 

print('Load data...')
train = pd.read_csv('/Users/evgenysidorov/Documents/Kaggle/bnp/data/train.csv')
target = train['target'].values
train = train.drop(['ID','target','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)
test = pd.read_csv('/Users/evgenysidorov/Documents/Kaggle/bnp/data/test.csv')
id_test = test['ID'].values
test = test.drop(['ID','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)

print('Clearing...')
num_vars = ['v1', 'v2', 'v4', 'v5', 'v6', 'v7', 'v9', 'v10', 'v11',
            'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20',
            'v21', 'v26', 'v27', 'v28', 'v29', 'v32', 'v33', 'v34', 'v35', 'v38',
            'v39', 'v40', 'v41', 'v42', 'v43', 'v44', 'v45', 'v48', 'v49', 'v50',
            'v55', 'v57', 'v58', 'v59', 'v60', 'v61', 'v62', 'v64', 'v65', 'v67',
            'v68', 'v69', 'v70', 'v72', 'v76', 'v77', 'v78', 'v80', 'v83', 'v84', 
            'v85', 'v86', 'v87', 'v88', 'v90', 'v93', 'v94', 'v96', 'v97', 'v98', 
            'v99', 'v100', 'v101', 'v102', 'v103', 'v104', 'v106', 'v111', 'v114',
            'v115', 'v120', 'v121', 'v122', 'v126', 'v127', 'v129', 'v130', 'v131']

vs = pd.concat([train, test])
for c in num_vars:
    if c not in train.columns:
        continue
    
    train.loc[train[c].round(5) == 0, c] = 0
    test.loc[test[c].round(5) == 0, c] = 0

    delimiter = find_delimiter(vs, c)
    train[c] *= 1/delimiter
    test[c] *= 1/delimiter

for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
    if train_series.dtype == 'O':
        #for objects: factorize
        train[train_name], tmp_indexer = pd.factorize(train[train_name])
        test[test_name] = tmp_indexer.get_indexer(test[test_name])
        #but now we have -1 values (NaN)
    else:
        #for int or float: fill NaN
        tmp_len = len(train[train_series.isnull()])
        if tmp_len>0:
            #print "mean", train_series.mean()
            train.loc[train_series.isnull(), train_name] = -999 
        #and Test
        tmp_len = len(test[test_series.isnull()])
        if tmp_len>0:
            test.loc[test_series.isnull(), test_name] = -999

# X_train = train
# X_test = test
# print('Training...')
# extc = ExtraTreesClassifier(n_estimators=1000,max_features= 50,criterion= 'entropy',min_samples_split= 4,
#                             max_depth= 35, min_samples_leaf= 2, n_jobs = -1)      

# extc.fit(X_train,target) 

# print('Predict...')
# y_pred = extc.predict_proba(X_test)
# #print y_pred

# pd.DataFrame({"ID": id_test, "PredictedProb": y_pred[:,1]}).to_csv('extra_trees.csv',index=False)

Load data...
Clearing...


/Users/evgenysidorov/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:16: FutureWarning: pd.rolling_apply is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=2).apply(func=<function>,kwargs=<dict>,args=<tuple>)


In [38]:
train.describe()

,v1,v2,v3,v4,v5,v6,v7,v9,v10,v11,...,v115,v120,v121,v122,v125,v126,v127,v129,v130,v131
count,1.143210e+05,1.143210e+05,114321.000000,1.143210e+05,114321.000000,1.143210e+05,1.143210e+05,1.143210e+05,114321.000000,1.143210e+05,...,1.143210e+05,1.143210e+05,1.143210e+05,1.143210e+05,114321.000000,1.143210e+05,1.143210e+05,114321.000000,1.143210e+05,1.143210e+05
mean,9.014050e+05,4.170910e+06,-0.027327,2.293265e+06,322.380684,1.333919e+06,1.359944e+06,5.042546e+06,85.265776,8.459191e+06,...,5.770817e+06,7.066876e+05,1.513387e+06,3.808906e+06,33.298134,8.573408e+05,1.791173e+06,0.310144,1.064414e+06,9.605825e+05
std,1.124711e+06,4.276121e+06,0.182573,2.185737e+06,1159.745602,1.252359e+06,1.271206e+06,4.661459e+06,70.146482,7.459733e+06,...,5.265066e+06,8.227550e+05,1.664306e+06,3.606545e+06,21.669189,8.290576e+05,1.978734e+06,0.693262,1.320708e+06,1.188915e+06
min,-9.990000e+02,-9.990000e+02,-1.000000,-9.990000e+02,-999.000000,-9.990000e+02,-9.990000e+02,-9.990000e+02,-999.000000,-9.990000e+02,...,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-1.000000,-9.990000e+02,-9.990000e+02,0.000000,-9.990000e+02,-9.990000e+02
25%,-9.990000e+02,-9.990000e+02,0.000000,-9.990000e+02,-999.000000,-9.990000e+02,-9.990000e+02,-9.990000e+02,48.004382,-9.990000e+02,...,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,16.000000,-9.990000e+02,-9.990000e+02,0.000000,-9.990000e+02,-9.990000e+02
50%,5.221852e+05,4.012424e+06,0.000000,2.701382e+06,997.290611,1.723745e+06,1.765225e+06,6.734065e+06,60.005470,1.413592e+07,...,8.303525e+06,5.386074e+05,1.355381e+06,4.718441e+06,29.000000,1.142534e+06,1.497287e+06,0.000000,8.312030e+05,5.767004e+05
75%,1.568062e+06,7.476733e+06,0.000000,4.255126e+06,1327.504218,2.412330e+06,2.454801e+06,9.210223e+06,96.008800,1.513970e+07,...,1.040675e+07,1.201306e+06,2.556375e+06,6.930692e+06,47.000000,1.509704e+06,3.107414e+06,0.000000,1.655111e+06,1.685048e+06
max,1.960784e+07,1.980198e+07,2.000000,1.960784e+07,2974.866890,1.941748e+07,1.941748e+07,1.980198e+07,847.077330,1.941748e+07,...,1.941748e+07,1.009152e+07,1.960784e+07,1.980198e+07,89.000000,1.421056e+07,1.960784e+07,11.000000,1.960784e+07,1.960784e+07


# Another example

In [88]:
import pandas as pd
import xgboost as xgb
import csv

# XGBoost params:
xgboost_params = { 
   "objective": "binary:logistic",
   "booster": "gbtree",
   "eval_metric": "logloss",
   "eta": 0.01, # 0.06, #0.01,
   #"min_child_weight": 240,
   "subsample": 0.75,
   "colsample_bytree": 0.68,
   "max_depth": 7
}

print('Load data...')
train = pd.read_csv('/Users/evgenysidorov/Documents/Kaggle/bnp/data/train.csv')
target = train['target']
train = train.drop(['ID','target'],axis=1)
test = pd.read_csv('/Users/evgenysidorov/Documents/Kaggle/bnp/data/test.csv')
ids = test['ID'].values
test = test.drop(['ID'],axis=1)
#
print('Clearing...')
for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
    if train_series.dtype == 'O':
        #for objects: factorize
        train[train_name], tmp_indexer = pd.factorize(train[train_name])
        test[test_name] = tmp_indexer.get_indexer(test[test_name])
        #but now we have -1 values (NaN)
    else:
        #for int or float: fill NaN
        tmp_len = len(train[train_series.isnull()])
        if tmp_len>0:
            train.loc[train_series.isnull(), train_name] = train_series.mean()
        #and Test
        tmp_len = len(test[test_series.isnull()])
        if tmp_len>0:
            test.loc[test_series.isnull(), test_name] = train_series.mean()  #TODO

for train_index, val_index in ss:
    xgtrain = xgb.DMatrix(tr.iloc[train_index, 2:].as_matrix(), label = tr.iloc[train_index, 1].values)
    xgtest = xgb.DMatrix(tr.iloc[val_index, 2:].as_matrix(), label = tr.iloc[val_index, 1].values)
    watchlist = [(xgtest, 'eval'), (xgtrain, 'train')]

#Now let's fit the model
    print('Fit the model...')
    boost_round = 1800 #CHANGE THIS BEFORE START
    clf = xgb.train(xgboost_params,xgtrain,num_boost_round=boost_round,verbose_eval=True,maximize=False, evals=watchlist)

    #Make predict
    print('Predict...')
#     test_preds = clf.predict(xgtest, ntree_limit=clf.best_iteration)
#     # Save results
#     #
#     predictions_file = open("simple_xgboost_result.csv", "w")
#     open_file_object = csv.writer(predictions_file)
#     open_file_object.writerow(["ID", "PredictedProb"])
#     open_file_object.writerows(zip(ids, test_preds))
#     predictions_file.close()
#     #
#     print('Done.')

Load data...
Clearing...
Fit the model...


[0]	eval-logloss:0.689879	train-logloss:0.689719
[1]	eval-logloss:0.686533	train-logloss:0.686402
[2]	eval-logloss:0.682867	train-logloss:0.682687
[3]	eval-logloss:0.679730	train-logloss:0.679461
[4]	eval-logloss:0.676380	train-logloss:0.676046
[5]	eval-logloss:0.672935	train-logloss:0.672534
[6]	eval-logloss:0.669706	train-logloss:0.669227
[7]	eval-logloss:0.666381	train-logloss:0.665851
[8]	eval-logloss:0.663298	train-logloss:0.662676
[9]	eval-logloss:0.660100	train-logloss:0.659428
[10]	eval-logloss:0.656943	train-logloss:0.656203
[11]	eval-logloss:0.653851	train-logloss:0.653044
[12]	eval-logloss:0.651158	train-logloss:0.650303
[13]	eval-logloss:0.648523	train-logloss:0.647608
[14]	eval-logloss:0.645741	train-logloss:0.644743
[15]	eval-logloss:0.643016	train-logloss:0.641933
[16]	eval-logloss:0.640185	train-logloss:0.639039
[17]	eval-logloss:0.637534	train-logloss:0.636327
[18]	eval-logloss:0.634966	train-logloss:0.633673
[19]	eval-logloss:0.632428	train-logloss:0.631061
[20]	eval-

KeyboardInterrupt: 